# Connect to drive and install dataset

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls

gdrive	sample_data


In [0]:
import urllib.request
print('Beginning file download with urllib2...')
url = 'https://storage.googleapis.com/hackaitn2019-datasets/computer-vision-expensya.zip'
urllib.request.urlretrieve(url, '/content/gdrive/My Drive/computer-vision-expensya.zip')

Beginning file download with urllib2...


('/content/gdrive/My Drive/computer-vision-expensya.zip',
 <http.client.HTTPMessage at 0x7ff9348242b0>)

In [0]:
import os
os.chdir('gdrive/My Drive/projet isid/data')  #change dir
  

In [0]:
!ls

csv  images  img_tmp  saved  test  test_out  test.zip  train  train_out


# preprocess images and  save in another folder

In [0]:
import cv2
from imutils import paths
import numpy as np
import os
try:
 from PIL import Image
except ImportError:
 import Image

In [0]:
import imutils

In [0]:
os.chdir('test')  #change dir


In [0]:

path="traint/train"
dist="train_out"


imagePaths = sorted(list(paths.list_images(path)))

i=0

for img_path in imagePaths:



    # resize each image conveniently
    im = Image.open(img_path)
    length_x, width_y = im.size
    factor = min(1, float(1024.0 / length_x))
    size = int(factor * length_x), int(factor * width_y)
    im_resized = im.resize(size, Image.ANTIALIAS)
    im_resized.save(img_path, dpi=(300, 300))


    img = cv2.imread(img_path, 0) 
    
    
    # Apply dilation and erosion to remove some noise
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)


    # Apply blur to smooth out the edges
    img = cv2.GaussianBlur(img, (5, 5), 0)
    
    # use a threshhol so that every pixel is 0 or 255 according to this threshhold
    th,img = cv2.threshold(img, 100, 255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)


    '''
    # find ticket contour
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edged = cv2.Canny(gray, 10, 50)
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if imutils.is_cv2() else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:5]
    screenCnt = None
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:
            screenCnt = approx
            break

    

    img=img[screenCnt[0]:screenCnt[1],screenCnt[2]:screenCnt[3]]
    '''

    
    # save img 
    name=img_path.split("/")[2].split(".")[0]
    cv2.imwrite(dist+"/"+name+".png", img)

    i+=1
    print(i)

# use pytesseract on the preproccessed images to extract their content

In [0]:
#!sudo apt install tesseract-ocr
#!pip install pytesseract

In [0]:
import pytesseract


In [0]:
path="train_out"
train_dict={}

imagePaths = sorted(list(paths.list_images(path)))

i=0
for img_path in imagePaths:

    extractedInformation = pytesseract.image_to_string(Image.open(img_path))

    id=int(img_path.split("/")[1].split(".")[0])
    train_dict[id]=extractedInformation

    i+=1
    print(i)


# add the text extracted from the tickets to the csv file 


In [0]:
import pandas as pd

In [0]:
os.chdir('../')  #change dir

!ls

csv  images  img_tmp  saved  test  test_out  test.zip  train  train_out


In [0]:
df = pd.read_csv ('testset_1.0.csv')
df.shape

(1227, 2)

In [0]:
df.columns = ['index','img_id','fuel','parking','restaurant','transport']

In [0]:
df.head()

,img_id,content
0,0,LINE\n‘TAX INVOICE\n\n3 Upper Pickering Street...
1,1,The 02\n\nUK-SE10 London\nTax code GB805124369...
2,10,Bennigan’s\n\nTrish - American\n\ntavern & Gri...
3,100,don Underground <@London Underground Lo\n\noso...
4,1000,VENCHT SPA\n\nCioccolato dal 1878\nAEROPORTO F...


In [0]:
df["content"] = df["img_id"].map(lambda row:train_dict[ row ] )



In [0]:
df = df.drop(columns=['index'], axis=1)

In [0]:
df.to_csv('saved/testset_2.0.csv', index=False) 


### remove noise for extracted text 

In [0]:
df = pd.read_csv ('saved/testset_2.0.csv')
df.shape

(1227, 2)

In [0]:
id_words={}

for i in range(len(df)):
  id=df["img_id"][i]
  words=df["content"][i]


  words=str(words)

  l=words.split("\n")

  id_words[id]=l

In [0]:
caracters=[' ',',',"'",   'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','é','ç','à','è','1','2','3','4','5','6','7','8','9']

for img in id_words: 
  l=[]

  for word in id_words[img]:

    remove=False

    if (len(word)<3 ):
      remove=True

    space=True
    for c in word:
      if c !=" ":
        space=False

    for c in word:
      if c !=caracters:
        remove=True

    if space :
      remove=True
            

    if not remove :
      l.append(word)

  id_words[img]=l




In [0]:
df["content"] = df["img_id"].map(lambda row:id_words[ row ] )



In [0]:
df.head()

,img_id,content
0,0,"[LINE, ‘TAX INVOICE, 3 Upper Pickering Street,..."
1,1,"[The 02, UK-SE10 London, Tax code GB805124369,..."
2,10,"[Bennigan’s, Trish - American, tavern & Grill,..."
3,100,"[don Underground <@London Underground Lo, osoc..."
4,1000,"[VENCHT SPA, Cioccolato dal 1878, AEROPORTO FI..."


In [0]:
'''
i=0
for img in id_words: 

  if(id_words[img]==['nan']):
    i=i+1



k=0
for i in range ( len(df)): 

  if(type(df["content"][i]) == float):
    k=k+1
'''

In [0]:
df.to_csv('saved/testset_2.2.csv', index=False) 


# add normal encoding and detect language

In [4]:
!ls

csv  images  img_tmp  saved  test  test_out  test.zip  train  train_out


In [0]:
dataset= pd.read_csv('saved/dataset_2.2.csv',delimiter=',')

In [10]:
dataset.head()

,img_id,fuel,parking,restaurant,transport,content
0,4209,0,1,0,0,"['1% erria So :', 'ike parcs équipés :', 'fom}..."
1,825,0,0,1,0,"['GRS. S.RL', 'STR. CEBROSA 166', '10036 - SET..."
2,2640,0,0,1,0,"['EJ ToT) 1', 'HYCHEF RIST COHH.S.p.A.', 'Aero..."
3,845,0,0,0,1,"['Trenord Sri', 'Obbligo di Convalida', 'Valid..."
4,915,0,0,0,1,"['Ary', 'INTERURBANO CON', 'VALIDITA” ORAFUA S..."


In [0]:
# one hot encoding to normal encoding
dataset.loc[dataset['fuel']==1,'target']=0
dataset.loc[dataset['parking']==1,'target']=1
dataset.loc[dataset['restaurant']==1,'target']=2
dataset.loc[dataset['transport']==1,'target']=3

In [0]:
dataset = dataset.dropna()

In [13]:
dataset.shape


(4020, 7)

In [0]:
def detectlang(text):
    lang=detect(text)
    return (lang)

In [0]:
#!pip install langdetect
#!pip install textblob
from langdetect import detect
l=[]
non_classified=[]
for i in dataset['content']:
    try:
        l.append(detect(i))
    except:
        dataset.drop(dataset[dataset.content==i ].index, inplace=True)



In [19]:
len(non_classified)


0

In [0]:
dataset['lang']=l

In [45]:
dataset.head()

,img_id,fuel,parking,restaurant,transport,content,target,lang
0,4209,0,1,0,0,"['1% erria So :', 'ike parcs équipés :', 'fom}...",1.0,en
1,825,0,0,1,0,"['GRS. S.RL', 'STR. CEBROSA 166', '10036 - SET...",2.0,en
2,2640,0,0,1,0,"['EJ ToT) 1', 'HYCHEF RIST COHH.S.p.A.', 'Aero...",2.0,ca
3,845,0,0,0,1,"['Trenord Sri', 'Obbligo di Convalida', 'Valid...",3.0,it
4,915,0,0,0,1,"['Ary', 'INTERURBANO CON', 'VALIDITA” ORAFUA S...",3.0,en


# data preprocessing 2.0 ; stop word, tokenozition and semming 


###### stopwords



In [22]:
dataset.lang.value_counts()

en    1678
de     579
fr     422
it     343
pt     339
tl     186
es     170
ca     141
ro      30
nl      28
no      14
et      11
so      10
vi       9
pl       6
hu       6
af       6
cy       5
sv       5
id       5
sl       3
fi       3
tr       2
sw       2
hr       2
sq       1
da       1
sk       1
cs       1
lt       1
Name: lang, dtype: int64

In [23]:
import nltk
nltk.download('stopwords')
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
print (stopwords.fileids())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['arabic', 'azerbaijani', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [0]:
stopswords=[]
for i in stopwords.fileids():
    stopswords.extend(stopwords.words(i))

###### create corpus after stop word, tokenization and stemming


In [0]:
corpus = []
for i in range(dataset.shape[0]):
    recipts = re.sub('[^a-zA-Z]', ' ', list(dataset['content'])[i])
    recipts = recipts.lower()
    recipts = recipts.split()
    ps = PorterStemmer()
    recipts = [ps.stem(word) for word in recipts if not word in set(stopswords)]
    recipts = ' '.join(recipts)
    corpus.append(recipts)




# Creating and running the Bag of Words models






In [0]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray()


In [27]:
X.shape

(4010, 27060)

## GaussianNB model

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix


In [29]:
y1=dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size = 0.20, random_state = 0)

# Fitting Naive Bayes to the Training set
classifier = GaussianNB()
classifier.fit(X_train, y_train)



GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

In [31]:
cm

array([[223,  16,  15,   8],
       [  9, 100,   7,   6],
       [  9,  10, 147,   8],
       [ 19,  23,  13, 189]])

In [32]:
a=0
b=0
for i in range(len(cm)):
    b+=cm[i][i]
    for j in range(len(cm[0])):
        a+=cm[i][j]
b/a

0.8216957605985037

##  deep model

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout

model = Sequential()

# Input layer and hidden layer 
model.add(Dense(activation="relu", input_dim=27060, units=700, kernel_initializer="uniform"))
model.add(Dropout(0.285))
model.add(Dense(activation="linear", input_dim=700, units=4, kernel_initializer="uniform"))


In [0]:

# Compile the ANN
model.compile(optimizer="adagrad", loss="mean_squared_error", metrics=["accuracy"])

In [0]:
y = dataset[['fuel','parking','restaurant','transport']]
X_train_deep, X_test_deep, y_train_deep, y_test_deep = train_test_split(X, y, test_size = 0.20, random_state = 0)


In [41]:
model.fit(X_train_deep, y_train_deep, epochs=100, batch_size=128)




Epoch 1/100





3208/3208 [==============================] - 8s 3ms/step - loss: 0.1258 - acc: 0.7696
Epoch 2/100
3208/3208 [==============================] - 7s 2ms/step - loss: 0.0636 - acc: 0.9408
Epoch 3/100
3208/3208 [==============================] - 7s 2ms/step - loss: 0.0322 - acc: 0.9610
Epoch 4/100
3208/3208 [==============================] - 7s 2ms/step - loss: 0.0247 - acc: 0.9660
Epoch 5/100
3208/3208 [==============================] - 7s 2ms/step - loss: 0.0221 - acc: 0.9651
Epoch 6/100
3208/3208 [==============================] - 7s 2ms/step - loss: 0.0202 - acc: 0.9670
Epoch 7/100
3208/3208 [==============================] - 7s 2ms/step - loss: 0.0188 - acc: 0.9685
Epoch 8/100
3208/3208 [==============================] - 7s 2ms/step - loss: 0.0182 - acc: 0.9691
Epoch 9/100
3208/3208 [==============================] - 7s 2ms/step - loss: 0.0173 - acc: 0.9673
Epoch 10/100
3208/3208 [==============================] - 7s 2ms/step - loss: 0.0166 - acc: 0.9685
Epoch 11/10

In [46]:
score = model.evaluate(X_test_deep, y_test_deep, batch_size=128)

802/802 [==============================] - 1s 626us/step


In [47]:
print(score)

[0.07247524665775441, 0.8528678295321]


In [0]:
pred=model.predict(X_test_deep)
